# 第9章: ベクトル空間法 (I)
enwiki-20150112-400-r10-105752.txt.bz2は，2015年1月12日時点の英語のWikipedia記事のうち，約400語以上で構成される記事の中から，ランダムに1/10サンプリングした105,752記事のテキストをbzip2形式で圧縮したものである．このテキストをコーパスとして，単語の意味を表すベクトル（分散表現）を学習したい．第9章の前半では，コーパスから作成した単語文脈共起行列に主成分分析を適用し，単語ベクトルを学習する過程を，いくつかの処理に分けて実装する．第9章の後半では，学習で得られた単語ベクトル（300次元）を用い，単語の類似度計算やアナロジー（類推）を行う．

なお，問題83を素直に実装すると，大量（約7GB）の主記憶が必要になる． メモリが不足する場合は，処理を工夫するか，1/100サンプリングのコーパスenwiki-20150112-400-r100-10576.txt.bz2を用いよ．

## 80. コーパスの整形
文を単語列に変換する最も単純な方法は，空白文字で単語に区切ることである． ただ，この方法では文末のピリオドや括弧などの記号が単語に含まれてしまう． そこで，コーパスの各行のテキストを空白文字でトークンのリストに分割した後，各トークンに以下の処理を施し，単語から記号を除去せよ．

- トークンの先頭と末尾に出現する次の文字を削除: .,!?;:()[]'"
- 空文字列となったトークンは削除

以上の処理を適用した後，トークンをスペースで連結してファイルに保存せよ．

In [1]:
!head -n 5 ../data/enwiki-20150112-400-r100-10576.txt

Anarchism

Anarchism is a political philosophy that advocates stateless societies often defined as self-governed voluntary institutions, but that several authors have defined as more specific institutions based on non-hierarchical free associations. Anarchism holds the state to be undesirable, unnecessary, or harmful. While anti-statism is central, anarchism entails opposing authority or hierarchical organisation in the conduct of human relations, including, but not limited to, the state system.
As a subtle and anti-dogmatic philosophy, anarchism draws on many currents of thought and strategy. Anarchism does not offer a fixed body of doctrine from a single particular world view, instead fluxing and flowing as a philosophy. There are many types and traditions of anarchism, not all of which are mutually exclusive. Anarchist schools of thought can differ fundamentally, supporting anything from extreme individualism to complete collectivism. Strains of anarchism have often been divided int

In [2]:
def tokenize(line):
    for token in line.split():
        yield token.strip('.,!?;:()[]\'"')

def knock_80():
    with open('../data/enwiki-20150112-400-r100-10576.txt') as f_in:
        with open('../work/enwiki-20150112-400-r100-10576-tokenized.txt', 'w') as f_out:
            for line in f_in:
                tokens = [cleaned_token for cleaned_token in tokenize(line) if cleaned_token]
                if len(tokens) > 0:
                    tokenized_sentence = ' '.join(tokens)
                    f_out.write(tokenized_sentence + '\n')

knock_80()

In [3]:
!head -n 5 ../work/enwiki-20150112-400-r100-10576-tokenized.txt

Anarchism
Anarchism is a political philosophy that advocates stateless societies often defined as self-governed voluntary institutions but that several authors have defined as more specific institutions based on non-hierarchical free associations Anarchism holds the state to be undesirable unnecessary or harmful While anti-statism is central anarchism entails opposing authority or hierarchical organisation in the conduct of human relations including but not limited to the state system
As a subtle and anti-dogmatic philosophy anarchism draws on many currents of thought and strategy Anarchism does not offer a fixed body of doctrine from a single particular world view instead fluxing and flowing as a philosophy There are many types and traditions of anarchism not all of which are mutually exclusive Anarchist schools of thought can differ fundamentally supporting anything from extreme individualism to complete collectivism Strains of anarchism have often been divided into the categories of

## 81. 複合語からなる国名への対処
英語では，複数の語の連接が意味を成すことがある．例えば，アメリカ合衆国は"United States"，イギリスは"United Kingdom"と表現されるが，"United"や"States"，"Kingdom"という単語だけでは，指し示している概念・実体が曖昧である．そこで，コーパス中に含まれる複合語を認識し，複合語を1語として扱うことで，複合語の意味を推定したい．しかしながら，複合語を正確に認定するのは大変むずかしいので，ここでは複合語からなる国名を認定したい．

インターネット上から国名リストを各自で入手し，80のコーパス中に出現する複合語の国名に関して，スペースをアンダーバーに置換せよ．例えば，"United States"は"United_States"，"Isle of Man"は"Isle_of_Man"になるはずである．

In [4]:
# 国名リストは以下のものを使用
# https://gist.github.com/kalinchernev/486393efcca01623b18d
!head -n 20 ../data/countries

Afghanistan
Albania
Algeria
Andorra
Angola
Antigua & Deps
Argentina
Armenia
Australia
Austria
Azerbaijan
Bahamas
Bahrain
Bangladesh
Barbados
Belarus
Belgium
Belize
Benin
Bhutan


In [5]:
from collections import defaultdict
import pprint

def read_compound_name_countries():
    with open('../data/countries') as f:
        compound_name_countries = defaultdict(list)
        for line in f:
            country_name_parts = line.rstrip().split(' ')
            if len(country_name_parts) > 1:
                head_token = country_name_parts[0]
                rest_token = country_name_parts[1:]
                # keyはheadの単語、valueは続く単語列のlist
                # 'United': [['Arab', 'Emirates'], ['Kingdom'], ['States']]
                # 'St': [['Kitts', '&', 'Nevis'], ['Lucia']]
                compound_name_countries[head_token].append(rest_token)
        return compound_name_countries
    
def replace_compound_name_country_in_a_line(line):
    replaced = []
    compound_name_countries = read_compound_name_countries()
    skip_counter = 0
    tokens = line.rstrip().split(' ')
    for i, token in enumerate(tokens):
        if skip_counter > 0:
            # 置換した複合語の単語数だけ読み飛ばす
            skip_counter -= 1
            continue
        if token in compound_name_countries:
            rest_token_list = compound_name_countries[token]
            for rest_token in rest_token_list:
                if tokens[i+1: i+len(rest_token)+1] == rest_token:
                    skip_counter = len(rest_token)
                    replaced.append('_'.join(tokens[i: i+len(rest_token)+1]))
        if skip_counter == 0:
            replaced.append(token)
    return ' '.join(replaced)
 
def knock_81():
    with open('../work/enwiki-20150112-400-r100-10576-tokenized.txt') as f_in:
        with open('../work/enwiki-20150112-400-r100-10576-compound_replaced.txt', 'w') as f_out:
            for line in f_in:
                f_out.write(replace_compound_name_country_in_a_line(line) + '\n')

knock_81()

In [6]:
!head -n 10 ../work/enwiki-20150112-400-r100-10576-compound_replaced.txt | grep United

The term anarchism is a compound word composed from the word anarchy and the suffix -ism themselves derived respectively from the Greek i.e anarchy from anarchos meaning one without rulers from the privative prefix ἀν- an- i.e without and archos i.e leader ruler cf archon or arkhē i.e authority sovereignty realm magistracy and the suffix or -ismos -isma from the verbal infinitive suffix -ίζειν -izein The first known use of this word was in 1539."Anarchist was the term adopted by Maximilien de Robespierre to attack those on the left whom he had used for his own ends during the French Revolution but was determined to get rid of though among these anarchists there were few who exhibited the social revolt characteristics of later anarchists There would be many revolutionaries of the early nineteenth century who contributed to the anarchist doctrines of the next generation such as William Godwin and Wilhelm Weitling but they did not use the word anarchist or anarchism in describing themselv

In [7]:
!head -n 1000 ../work/enwiki-20150112-400-r100-10576-compound_replaced.txt | grep Papua

The British Museum's Oceanic collections originate from the vast area of the Pacific Ocean stretching from Papua_New_Guinea to Easter Island from New_Zealand to Hawaii The three main anthropological groups represented in the collection are Polynesia Melanesia and Micronesia – Aboriginal art from Australia is considered separately in its own right Metal working was not indigenous to Oceania before Europeans arrived so many of the artefacts from the collection are made from stone shell bone and bamboo The British Museum is fortunate in having some of the earliest objects made in Oceania in its collections many of which were collected by members of Cook's and Vancouver's expeditions before Western culture significantly impacted on local practices and ways of thinking The Wilson cabinet of curiosities from Palau is another example of pre-contact ware A particularly important group of objects was purchased from the London Missionary Society in 1911 that includes the unique statue of A'a fro

## 82. 文脈の抽出
81で作成したコーパス中に出現するすべての単語$t$に関して，単語$t$と文脈語$c$のペアをタブ区切り形式ですべて書き出せ．ただし，文脈語の定義は次の通りとする．

- ある単語$t$の前後$d$単語を文脈語$c$として抽出する（ただし，文脈語に単語tそのものは含まない）
- 単語$t$を選ぶ度に，文脈幅$d$は${1,2,3,4,5}$の範囲でランダムに決める．

In [8]:
import random

def knock_82():
    with open('../work/enwiki-20150112-400-r100-10576-compound_replaced.txt') as f_in:
        with open('../work/enwiki-20150112-400-r100-10576-collocation.txt', 'w') as f_out:
            random.seed(0)
            for line in f_in:
                tokens = line.rstrip().split(' ')
                for i, token in enumerate(tokens):
                    context_words = []
                    d = random.randint(1, 5)
                    context_words += tokens[max(0, i-d): i]
                    context_words += tokens[i+1: i+1+d]
                    for context_word in context_words:
                        f_out.write('{}\t{}\n'.format(token, context_word))

knock_82()

In [9]:
!head -n 30 ../work/enwiki-20150112-400-r100-10576-collocation.txt

Anarchism	is
Anarchism	a
Anarchism	political
Anarchism	philosophy
is	Anarchism
is	a
a	Anarchism
a	is
a	political
a	philosophy
a	that
political	Anarchism
political	is
political	a
political	philosophy
political	that
political	advocates
political	stateless
political	societies
philosophy	Anarchism
philosophy	is
philosophy	a
philosophy	political
philosophy	that
philosophy	advocates
philosophy	stateless
philosophy	societies
that	is
that	a
that	political


## 83. 単語／文脈の頻度の計測
82の出力を利用し，以下の出現分布，および定数を求めよ．

- $f(t,c)$: 単語$t$と文脈語$c$の共起回数
- $f(t,\ast)$: 単語$t$の出現回数
- $f(\ast,c)$: 文脈語$c$の出現回数
- $N$: 単語と文脈語のペアの総出現回数

In [10]:
from collections import defaultdict
from tqdm import tqdm
import pickle

def knock_83():
    with open('../work/enwiki-20150112-400-r100-10576-collocation.txt', 'r') as f:
        total_count = 0
        t_counter = defaultdict(int)
        c_counter = defaultdict(int)
        tc_counter = defaultdict(int)
        for i, line in tqdm(enumerate(f)):
            t, c = line.lower().rstrip().split('\t')
            total_count += 1
            t_counter[t] += 1
            c_counter[c] += 1
            tc_counter[t + '\t' + c] += 1
        with open('../work/t_counter.pkl', 'wb') as f:
            pickle.dump(t_counter, f)
        with open('../work/c_counter.pkl', 'wb') as f:
            pickle.dump(c_counter, f)
        with open('../work/tc_counter.pkl', 'wb') as f:
            pickle.dump(tc_counter, f)
        print('N: ' + str(total_count))
    
knock_83()

68094461it [02:03, 552263.34it/s]


N: 68094461


## 84. 単語文脈行列の作成
83の出力を利用し，単語文脈行列Xを作成せよ．ただし，行列Xの各要素$X_{tc}$は次のように定義する．

- $ f(t,c) \geq 10 $ ならば，$ X_{tc} = \textrm{PPMI}(t,c) = \max \left\{ \log \frac {N \times f(t,c)}{f(t,\ast) \times f(\ast,c)}, 0 \right\} $
- $ f(t,c)<10 $ ならば，$ X_{tc} = 0 $

ここで，$\textrm{PPMI}(t,c)$はPositive Pointwise Mutual Information（正の相互情報量）と呼ばれる統計量である．なお，行列$X$の行数・列数は数百万オーダとなり，行列のすべての要素を主記憶上に載せることは無理なので注意すること．幸い，行列$X$のほとんどの要素は$0$になるので，非$0$の要素だけを書き出せばよい．

In [11]:
from scipy.sparse import lil_matrix
import math

N = 68094461

def knock_84():
    with open('../work/t_counter.pkl', 'rb') as f:
        t_counter = pickle.load(f)
    with open('../work/c_counter.pkl', 'rb') as f:
        c_counter = pickle.load(f)
    with open('../work/tc_counter.pkl', 'rb') as f:
        tc_counter = pickle.load(f)

    word_to_index = { t:i for i, t in enumerate(t_counter) }
    x = lil_matrix((len(t_counter), len(c_counter)))
    for tc, tc_count in tc_counter.items():
        if tc_count < 10:
            continue
        t, c = tc.split('\t')
        ppmi = max(math.log((N * tc_count) / (t_counter[t] * c_counter[c])), 0)
        t_index = word_to_index[t]
        c_index = word_to_index[c]
        x[t_index, c_index] = ppmi
    
    with open('../work/x_ppmi.pkl', 'wb') as f:
        pickle.dump(x, f)
    with open('../work/word_to_index.pkl', 'wb') as f:
        pickle.dump(word_to_index, f)

knock_84()

## 85. 主成分分析による次元圧縮
84で得られた単語文脈行列に対して，主成分分析を適用し，単語の意味ベクトルを300次元に圧縮せよ．

In [12]:
import pickle
from sklearn.decomposition import SparsePCA, TruncatedSVD

def knock_85():
    with open('../work/x_ppmi.pkl', 'rb') as f:
        x_ppmi = pickle.load(f)
    
    pca = TruncatedSVD(n_components=300)
    word_vector = pca.fit_transform(x_ppmi)
   
    with open('../work/word_vector.pkl', 'wb') as f:
        pickle.dump(word_vector, f)
    
knock_85()

## 86. 単語ベクトルの表示
85で得た単語の意味ベクトルを読み込み，"United States"のベクトルを表示せよ．ただし，"United States"は内部的には"United_States"と表現されていることに注意せよ．

In [13]:
import pickle

def knock_86():
    with open('../work/word_vector.pkl', 'rb') as f:
        word_vector = pickle.load(f)
    with open('../work/word_to_index.pkl', 'rb') as f:
        word_to_index = pickle.load(f)
    print(word_vector[word_to_index['United_States'.lower()]])

knock_86()

[ 4.18841405e+00 -1.16733813e+00  5.88192661e-01 -5.31552149e+00
 -2.73868465e+00 -5.14571637e-01 -3.51967080e+00 -3.11619359e-01
 -8.00750192e-02 -3.00999544e-02  5.35869599e+00  2.96782521e-01
  5.28023090e-01 -2.07343719e+00  1.51945365e+00 -9.80887213e-02
 -4.94769541e-01 -1.37029820e+00 -2.93637891e-01  1.42807161e+00
  1.42390976e+00  1.48054472e+00 -1.09191297e+00  1.55293497e-01
  9.95203580e-01 -5.30881693e-01 -1.60740449e+00 -4.63751760e-01
  2.45274238e+00 -2.49901007e-01  3.49011004e+00 -4.33954615e-02
 -1.44739058e-01  6.39001089e-01  6.63234649e-01 -2.53717827e-01
  5.31502746e-01  6.19995565e-02  1.32794942e+00  9.08111345e-01
  1.00661888e+00  7.42727174e-01 -2.93903376e-02  3.08937065e-01
  2.74166276e-01 -1.30062537e+00  1.92870932e+00 -6.66868005e-01
 -6.44848929e-01  2.64025875e-01 -4.08787430e-01  5.10416120e-01
 -1.89578034e+00 -2.50026319e-01 -2.94490330e-01  4.53999684e-01
  1.07606100e+00 -8.09314650e-01  1.87310839e-01 -9.76229163e-01
 -3.10846159e-01  1.97882

## 87. 単語の類似度
85で得た単語の意味ベクトルを読み込み，"United States"と"U.S."のコサイン類似度を計算せよ．ただし，"U.S."は内部的に"U.S"と表現されていることに注意せよ．

In [14]:
import pickle
from sklearn.metrics.pairwise import cosine_similarity

def knock_87():
    with open('../work/word_vector.pkl', 'rb') as f:
        word_vector = pickle.load(f)
    with open('../work/word_to_index.pkl', 'rb') as f:
        word_to_index = pickle.load(f)
        
    x = word_vector[word_to_index['United_States'.lower()]]
    y = word_vector[word_to_index['U.S'.lower()]]
    print(cosine_similarity([x],[y])[0][0])

knock_87()

0.829616260585497


## 88. 類似度の高い単語10件
85で得た単語の意味ベクトルを読み込み，"England"とコサイン類似度が高い10語と，その類似度を出力せよ．

In [15]:
import pickle
from sklearn.metrics.pairwise import cosine_similarity
import pprint

def get_similar_words_with_vector(target_vector, top_n, word_vectors, word_to_index, exclude_words=None):
    if not exclude_words:
        exclude_words = ()
    cosine_similarities = { word:cosine_similarity([target_vector],[word_vectors[index]])[0][0] for word,index in word_to_index.items() if word != exclude_words}
    return sorted(cosine_similarities.items(), key=lambda x:x[1], reverse=True)[0:top_n]
    
def get_similar_words(target_word, top_n):
    with open('../work/word_vector.pkl', 'rb') as f:
        word_vectors = pickle.load(f)
    with open('../work/word_to_index.pkl', 'rb') as f:
        word_to_index = pickle.load(f)
    target_vector = word_vectors[word_to_index[target_word.lower()]]
    return get_similar_words_with_vector(target_vector, top_n, word_vectors, word_to_index, (target_word.lower()))

def knock_88():
    pprint.pprint(get_similar_words("England", 10))
    
knock_88()

[('scotland', 0.7315167033166496),
 ('australia', 0.6475282415285533),
 ('wales', 0.608772335595254),
 ('spain', 0.6068454168091277),
 ('france', 0.5842340500667358),
 ('italy', 0.5777101779948076),
 ('ireland', 0.5754315996341104),
 ('germany', 0.566933012313069),
 ('united_kingdom', 0.5372928727506192),
 ('japan', 0.52293520211957)]


## 89. 加法構成性によるアナロジー
85で得た単語の意味ベクトルを読み込み，vec("Spain") - vec("Madrid") + vec("Athens")を計算し，そのベクトルと類似度の高い10語とその類似度を出力せよ．

In [16]:
import pickle
from sklearn.metrics.pairwise import cosine_similarity
import pprint

def knock_89():
    with open('../work/word_vector.pkl', 'rb') as f:
        word_vectors = pickle.load(f)
    with open('../work/word_to_index.pkl', 'rb') as f:
        word_to_index = pickle.load(f)
    spain_vector = word_vectors[word_to_index['Spain'.lower()]]
    madrid_vector = word_vectors[word_to_index['Madrid'.lower()]]
    athens_vector = word_vectors[word_to_index['Athens'.lower()]]
    target_vector = spain_vector - madrid_vector + athens_vector
    pprint.pprint(get_similar_words_with_vector(target_vector, 10, word_vectors, word_to_index))
    
knock_89()

[('spain', 0.882086888194791),
 ('sweden', 0.8430648590532241),
 ('italy', 0.8199000557649473),
 ('germany', 0.8155642697586805),
 ('austria', 0.8099618510512393),
 ('denmark', 0.7961119683916021),
 ('netherlands', 0.7801191670399344),
 ('belgium', 0.7740889027588628),
 ('finland', 0.7582261651974795),
 ('télévisions', 0.7465486148512578)]
